In [1]:
%matplotlib inline
import math
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [2]:
batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

## 初始化模型参数

In [3]:
def get_params(vocab_size, num_hiddens, device):
    dim_inputs=dim_outputs=vocab_size
    
    #输出正态分布随机初始化参数
    def normal(shape):
        # 在深度学习中，常常需要对权重进行初始化。
        # 乘以0.01是一种常见的权重初始化方法之一。
        # 通过将权重初始化为较小的值，可以帮助避免梯度消失或爆炸的问题，
        # 并且可以更快地收敛到较好的解决方案。
        # 乘以0.01可以将权重的数值范围缩小，
        # 从而使得激活函数的输入在其敏感区域内，
        # 有助于提高模型的效果和稳定性
        return torch.randn(size=shape, device=device) * 0.01
    # hidden layer parameter
    W_xh = normal((dim_inputs, num_hiddens))
    W_hh = normal((num_hiddens, num_hiddens))
    b_h = torch.zeros(num_hiddens, device=device)
    # output layer parameter
    W_hq = normal((num_hiddens, dim_outputs))
    b_q = torch.zeros(dim_outputs, device=device)
    # attach gradient
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

当time step为0的时候，没有t-1层的隐状态用来计算。因此直接把第0层的隐状态参数定义出来。

In [4]:
def rnn_init_hidden_state(batch_size, num_hiddens, device):
    #返回一个元组，方便之后实现其他模型使用
    return (torch.zeros((batch_size, num_hiddens), device=device), )

## 定义rnn forward function

In [5]:
def rnn_forward_fn(inputs, state, params):
    # inputs的形状：(时间步数量，批量大小，词表大小)
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    # 对每一个时间步进行运算
    for X in inputs:
        H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h)
        Y = torch.mm(H, W_hq) + b_q
        outputs.append(Y)
        
    # 返回的维度为（n*T，vocab_size）
    return torch.cat(outputs, dim=0), (H,)
        

## 定义RNN

In [9]:
class RNNModelScratch:
    def __init__(self,vocab_size, num_hiddens,get_params, device, init_state, forward_fn):
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.params = get_params(vocab_size,num_hiddens, device)
        self.init_state , self.forward_fn = init_state, forward_fn
    
    def __call__(self, X, state):
        # X:n(batchsize)*T(num_steps),X.T是为了把time-step放到axis=0的位置上方便后续的遍历
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        return self.forward_fn(X, state, self.params)
    
    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)

## 检查输出维度

In [10]:
X = torch.arange(10).reshape((2, 5))
num_hiddens = 512
net = RNNModelScratch(len(vocab), num_hiddens, get_params, d2l.try_gpu(),
                      rnn_init_hidden_state, rnn_forward_fn)
state = net.begin_state(X.shape[0], d2l.try_gpu())
Y, new_state = net(X.to(d2l.try_gpu()), state)
Y.shape, len(new_state), new_state[0].shape

(torch.Size([10, 28]), 1, torch.Size([2, 512]))

# 编写预测函数

In [34]:
def predict_chapter8(prefix, num_predicts, net, vocab, device):
    '''在prefix之后生成num_predicts个新字符'''
    # 先把state设置为初始化,因为是对prefix这一个短字符串进行预测，所以batch设为1
    state = net.begin_state(batch_size=1, device=device)
    # 输出的第一项设为prefix第一项在vocab中的索引
    outputs = [vocab[prefix[0]]]
    # 获取每个time step模型的输入
    get_input = lambda: torch.tensor([outputs[-1]],device=device).reshape((1, 1))
    # 预热模型，根据用户提供的前缀来调节语言模型
    for y in prefix[1:]:
        _, state = net(get_input(), state)
        outputs.append(vocab[y])
    for _ in range(num_predicts):  # 预测num_preds步
        y, state = net(get_input(), state)
        print(y.shape)
        # argmax函数返回y中每一行的最大值的索引。
        # 参数dim=1表示在第1维度上进行操作，即对每一行进行操作。
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    return ''.join([vocab.idx_to_token[i] for i in outputs])

In [35]:
predict_chapter8('time traveller ', 10, net, vocab, d2l.try_gpu())

torch.Size([1, 28])
torch.Size([1, 28])
torch.Size([1, 28])
torch.Size([1, 28])
torch.Size([1, 28])
torch.Size([1, 28])
torch.Size([1, 28])
torch.Size([1, 28])
torch.Size([1, 28])
torch.Size([1, 28])


'time traveller           '

# 编写训练函数

## 梯度裁剪 grad clipping

### 主要公式:

**$$\mathbf{g} \leftarrow \min\left(1, \frac{\theta}{\|\mathbf{g}\|}\right) \mathbf{g}.$$**

对于长度为$T$的序列，我们在迭代中计算这$T$个时间步上的梯度，
将会在反向传播过程中产生长度为$\mathcal{O}(T)$的矩阵乘法链。
如 :numref:`sec_numerical_stability`所述，
当$T$较大时，它可能导致数值不稳定，
例如可能导致梯度爆炸或梯度消失。
因此，循环神经网络模型往往需要额外的方式来支持稳定训练。

一般来说，当解决优化问题时，我们对模型参数采用更新步骤。
假定在向量形式的$\mathbf{x}$中，
或者在小批量数据的负梯度$\mathbf{g}$方向上。
例如，使用$\eta > 0$作为学习率时，在一次迭代中，
我们将$\mathbf{x}$更新为$\mathbf{x} - \eta \mathbf{g}$。
如果我们进一步假设目标函数$f$表现良好，
即函数$f$在常数$L$下是*利普希茨连续的*（Lipschitz continuous）。
也就是说，对于任意$\mathbf{x}$和$\mathbf{y}$我们有：

$$|f(\mathbf{x}) - f(\mathbf{y})| \leq L \|\mathbf{x} - \mathbf{y}\|.$$

在这种情况下，我们可以安全地假设：
如果我们通过$\eta \mathbf{g}$更新参数向量，则

$$|f(\mathbf{x}) - f(\mathbf{x} - \eta\mathbf{g})| \leq L \eta\|\mathbf{g}\|,$$

这意味着我们不会观察到超过$L \eta \|\mathbf{g}\|$的变化。
这既是坏事也是好事。
坏的方面，它限制了取得进展的速度；
好的方面，它限制了事情变糟的程度，尤其当我们朝着错误的方向前进时。

有时梯度可能很大，从而优化算法可能无法收敛。
我们可以通过降低$\eta$的学习率来解决这个问题。
但是如果我们很少得到大的梯度呢？
在这种情况下，这种做法似乎毫无道理。
一个流行的替代方案是通过将梯度$\mathbf{g}$投影回给定半径
（例如$\theta$）的球来裁剪梯度$\mathbf{g}$。
如下式：

(**$$\mathbf{g} \leftarrow \min\left(1, \frac{\theta}{\|\mathbf{g}\|}\right) \mathbf{g}.$$**)

通过这样做，我们知道梯度范数永远不会超过$\theta$，
并且更新后的梯度完全与$\mathbf{g}$的原始方向对齐。
它还有一个值得拥有的副作用，
即限制任何给定的小批量数据（以及其中任何给定的样本）对参数向量的影响，
这赋予了模型一定程度的稳定性。
梯度裁剪提供了一个快速修复梯度爆炸的方法，
虽然它并不能完全解决问题，但它是众多有效的技术之一。

下面我们定义一个函数来裁剪模型的梯度，
模型是从零开始实现的模型或由高级API构建的模型。
我们在此计算了所有模型参数的梯度的范数。


In [2]:
def grad_clipping(net, theta):
    '''同时兼容从零开始实现和简洁实现的RNN'''
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
        
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm 

## 训练

**注意事项：**

1. 对序列化的训练数据的不同采样方法（随机采样和顺序分区）将导致隐状态初始化的差异。
1. 我们在**更新模型参数**之前裁剪梯度。这样的操作的目的是，即使训练过程中某个点上发生了梯度爆炸，也能保证模型不会发散。
1. 我们用困惑度来评价模型。这样的度量确保了不同长度的序列具有可比性。

**困惑度：**$$ exp\left(-\frac{1}{n} \sum_{t=1}^{n}logP(x_t|x_{t-1}, \ldots, x_1)\right)$$

具体来说，当使用顺序分区(`use_random_iter = False`)时，
我们只在每个迭代周期的开始位置初始化隐状态。
由于下一个小批量数据中的第$i$个子序列样本
与当前第$i$个子序列样本相邻，
因此当前小批量数据最后一个样本的隐状态，
将用于初始化下一个小批量数据第一个样本的隐状态。
这样，存储在隐状态中的序列的历史信息
可以在一个迭代周期内流经相邻的子序列。
然而，在任何一点隐状态的计算，
都依赖于同一迭代周期中前面所有的小批量数据，
这使得梯度计算变得复杂。
为了降低计算量，在处理任何一个小批量数据之前，
我们先分离梯度，使得隐状态的梯度计算总是限制在一个小批量数据的时间步内。

当使用随机抽样(`use_random_iter = True`)时，因为每个样本都是在一个随机位置抽样的，
因此需要为每个迭代周期重新初始化隐状态。
`updater`是更新模型参数的常用函数。
它既可以是从头开始实现的`d2l.sgd`函数，
也可以是深度学习框架中内置的优化函数。

In [2]:
def train_epoch_chapter8(net, train_iter, loss, updater, device, use_random_iter):
    state, timer = None, d2l.Timer()
    metric = d2l.Accumulator(2)  # 训练损 失之和，token数量
    for X, Y in train_iter:
        if state is None or use_random_iter:
            # 如使用随机迭代，需要在每个小批量更新前初始化隐藏状态
            state = net.begin_state(batch_size=X.shape[0], device=device)
        else:
            if isinstance(net, nn.Module) and not isinstance(state, tuple):
                # 如使用相邻批量采样，且模型为PyTorch模型，需将隐藏状态从计算图分离
                state.detach_()
            else:
                # 使用相邻批量采样，且模型为自定义的模型(返回的类型为元组)
                for s in state:
                    s.detach_()
        y = Y.T.reshape(-1)  # 转置后把time-step放到axis=0的位置上方便后续的遍历，然后拉直成一个向量
        X, y = X.to(device), y.to(device)
        y_hat, state = net(X, state)
        l = loss(y_hat, y.long()).mean()  # y和y_hat都是拉直的向量，所以这里的loss计算方式跟MLP一样
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            grad_clipping(net, 1)
            updater.step()
        else:
            l.backward()
            grad_clipping(net, 1)
            # 因为已经调用了mean函数
            updater(batch_size=1)

        metric.add(l * y.numel(), y.numel())
        # mertric[0]是损失之和，metric[1]是token数量,二者相除就是交叉熵损失
    return math.exp(metric[0] / metric[1]), metric[1] / timer.stop()

In [ ]:
def train_chapter8(
    net, train_iter, vocab, lr, num_epochs, device, use_random_iter=False
):
    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(
        xlabel="epoch", ylabel="perplexity", legend=["train"], xlim=[10, num_epochs]
    )
    # 初始化更新函数
    if isinstance(net, nn.Module):
        updater = torch.optim.SGD(net.parameters(), lr)
    else:
        updater = lambda batch_size: d2l.sgd(net.params, lr, batch_size)
    # 定义预测函数
    predict = lambda prefix: predict_chapter8(prefix, 50, net, vocab, device)
    # 训练模型
    for epoch in range(num_epochs):
        perplexity, speed = train_epoch_chapter8(
            net, train_iter, loss, updater, device, use_random_iter
        )
        if (epoch + 1) % 10 == 0:
            print(predict("time traveller"))
            animator.add(epoch + 1, [perplexity])
    print(f'困惑度 {perplexity:.1f}, {speed:.1f} tokens/秒 {str(device)}')
    print(predict("time traveller"))
    print(predict("traveller"))

# 开始训练

In [ ]:
num_epochs, lr = 500, 1
train_chapter8(net, train_iter, vocab, lr, num_epochs, d2l.try_gpu())